In [25]:
import pandas as pd

import numpy as np

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans 

import folium # map rendering library


!wget -q -O 'canada_data.json' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


In [2]:
#with open('canada_data.json') as json_data:
    #canada_data = json.load(json_data)

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')  # all tables on the webpage are saved into the dataframe


In [19]:
postal_df = df[0] #using only the 1st table
postal_df = postal_df[postal_df.Borough != 'Not assigned']  #remove cells with "Not assigned as the borough value"

i = len(postal_df)
j = 0
while j < i:
    if(postal_df['Neighbourhood'].iloc[j] == 'Not assigned'):                #check cells under neighbourhood column that have not assigned 
        postal_df['Neighbourhood'].iloc[j] = postal_df['Borough'].iloc[j]   #replace "not assigned" with value in borough
    j = j + 1

postal_df = postal_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join)  #group dataframe based on postcode and borough; used apply and join to add commas
postal_df = postal_df.reset_index() #reset_index adds row for 0 to 102

print(postal_df.shape)
postal_df

(103, 3)


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [20]:
!wget -q -O 'LatLng.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

df_LatLng = pd.read_csv('LatLng.csv')
df_LatLng.head()

postal_df['Latitude'] = df_LatLng['Latitude']
postal_df['Longitude'] = df_LatLng['Longitude']
postal_df.head()

Data downloaded!


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [26]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(postal_df['Latitude'], postal_df['Longitude'], postal_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [23]:
Latitude = list(postal_df['Latitude'])
Longitude = list(postal_df['Longitude'])
#postal_df['Neighbourhood']